# Training model

In this notebook:

1.   Data preparation
2.   Download Xception model with ImageNet weights
3.   Add top layers for binary classification task on top of Xception
4.   Short train with Xception layers frozen
5.   Long train all layers until best validation loss value found, using early stopping



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import IPython.display as display
from PIL import Image
import matplotlib.pyplot as plt
import os
import pathlib

In [0]:
#Paths to the dataset
train_dir = '/content/gdrive/My Drive/dataset/1_train'
train_dir = pathlib.Path(train_dir)

valid_dir = '/content/gdrive/My Drive/dataset/2_validation'
valid_dir = pathlib.Path(valid_dir)

#Inspect number of photos in each subset
train_count = len(list(train_dir.glob('*/*.png')))
valid_count = len(list(valid_dir.glob('*/*.png')))
print("Number of training photos:", train_count, "\nNumber of validation photos:", valid_count)

Number of training photos: 5600 
Number of validation photos: 800


In [0]:
CLASS_NAMES = np.array(['bad', 'good'])
CLASS_NAMES[0]

'bad'

In [0]:
#This batch size provides same size for every batch
#Image dimensions required for the input to Xception network
BATCH_SIZE = 32
IMG_HEIGHT = 229
IMG_WIDTH = 229

#Steps for training and validation purposes
STEPS_PER_EPOCH = np.ceil(train_count/BATCH_SIZE)
STEPS_PER_EPOCH_V = np.ceil(valid_count/BATCH_SIZE)

In [0]:
#Train batches generator
train_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.xception.preprocess_input)

train_generator = train_datagen.flow_from_directory(directory=str(train_dir),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                    class_mode='binary')

Found 5600 images belonging to 2 classes.


In [0]:
#Validation batches generator
valid_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.xception.preprocess_input)

valid_generator = valid_datagen.flow_from_directory(directory=str(valid_dir),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                    class_mode='binary')

Found 800 images belonging to 2 classes.


In [0]:
#Import Xception model with weights trained on ImageNet dataset, without last layer
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(2, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [0]:
#Freezing all Xception layers for first 5 epochs
#Unfreezed remain only avg and output layer
for layer in base_model.layers:
    layer.trainable = False

checkpoint_cb = keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/image_x_model_ts.h5", save_best_only=True)
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_generator,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_generator,
                    validation_steps=STEPS_PER_EPOCH_V,
                    epochs=5,
                    callbacks=[checkpoint_cb])

Epoch 1/5
175/175 [==============================] - 55s 312ms/step - loss: 1.6746 - accuracy: 0.7782 - val_loss: 1.1073 - val_accuracy: 0.8625
Epoch 2/5
175/175 [==============================] - 54s 308ms/step - loss: 0.9850 - accuracy: 0.8061 - val_loss: 0.6902 - val_accuracy: 0.8175
Epoch 3/5
175/175 [==============================] - 53s 302ms/step - loss: 0.6226 - accuracy: 0.8300 - val_loss: 0.7437 - val_accuracy: 0.7550
Epoch 4/5
175/175 [==============================] - 54s 309ms/step - loss: 0.4637 - accuracy: 0.8429 - val_loss: 0.6020 - val_accuracy: 0.8562
Epoch 5/5
175/175 [==============================] - 53s 300ms/step - loss: 0.3974 - accuracy: 0.8525 - val_loss: 0.6853 - val_accuracy: 0.7550


In [0]:
#Unfreezing base model layers for next epochs
for layer in base_model.layers:
    layer.trainable = True

checkpoint_cb = keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/image_x_model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_generator,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_generator,
                    validation_steps=STEPS_PER_EPOCH_V,
                    epochs=100,
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
175/175 [==============================] - 151s 864ms/step - loss: 0.3171 - accuracy: 0.8691 - val_loss: 0.1708 - val_accuracy: 0.9275
Epoch 2/100
175/175 [==============================] - 144s 820ms/step - loss: 0.1370 - accuracy: 0.9379 - val_loss: 0.1052 - val_accuracy: 0.9575
Epoch 3/100
175/175 [==============================] - 143s 819ms/step - loss: 0.0688 - accuracy: 0.9704 - val_loss: 0.0581 - val_accuracy: 0.9775
Epoch 4/100
175/175 [==============================] - 141s 808ms/step - loss: 0.0473 - accuracy: 0.9780 - val_loss: 0.1028 - val_accuracy: 0.9613
Epoch 5/100
175/175 [==============================] - 143s 819ms/step - loss: 0.0415 - accuracy: 0.9816 - val_loss: 0.0524 - val_accuracy: 0.9775
Epoch 6/100
175/175 [==============================] - 141s 806ms/step - loss: 0.0330 - accuracy: 0.9862 - val_loss: 0.0875 - val_accuracy: 0.9712
Epoch 7/100
175/175 [==============================] - 139s 794ms/step - loss: 0.0269 - accuracy: 0.9868 - val_loss: 0